<a href="https://colab.research.google.com/github/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decoding_109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/109/109_Data_Final.csv?raw=true

!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv
!mv 109_Data_Final.csv\?raw\=true 109_Data_Final.csv

--2019-04-17 08:16:40--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-17 08:16:40--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-17 08:16:40--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [3]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('109_Data_Final.csv')
df.set_index('Subject')
df.head()

df_109 = pd.read_csv('109_Data_Final.csv')
df_109.set_index('Subject')

,Unnamed: 0,Block,Trial,Time,O2,O1,OZ,PZ,P4,CP4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
Subject,,,,,,,,,,,,,,,,,,,,,
109,0,1,1,-200,-2.4540,-7.4055,-3.7317,1.9958,-0.3961,6.4609,...,9.8221,12.2913,16.4498,5.1061,14.7567,39.3748,14.5422,10.6530,7.2805,7.0
109,1,1,1,-196,-3.7176,-10.9205,-5.6706,3.7010,-0.5067,5.8601,...,8.0161,10.2671,14.9112,5.1602,14.8205,29.3318,14.7717,17.7262,5.2050,7.0
109,2,1,1,-192,-6.3099,-14.5290,-8.9660,4.2277,-1.1666,4.9355,...,4.7711,6.3217,11.6578,1.8895,11.7379,31.0242,6.2631,17.7477,1.9524,7.0
109,3,1,1,-188,-7.3233,-13.6481,-9.2608,5.3117,-0.7020,4.9301,...,2.0940,2.9055,9.4231,-0.8641,8.4070,17.0047,9.1701,15.7770,-6.2897,7.0
109,4,1,1,-184,-6.6049,-10.7886,-7.4505,6.4961,-0.3101,4.8077,...,0.7413,1.2050,9.1359,-2.1851,7.0578,31.5447,28.4067,13.9127,13.4890,7.0
109,5,1,1,-180,-5.8761,-9.4650,-6.4137,6.0440,-1.6662,3.2746,...,0.0737,0.5549,8.2314,-2.8626,6.6793,20.5486,5.9248,11.7926,-9.2164,7.0
109,6,1,1,-176,-4.9485,-7.1693,-5.2465,5.2205,-2.5981,2.5221,...,0.3043,0.7138,6.1935,-2.9951,5.9663,1.3211,-10.3782,7.0524,-14.9233,7.0
109,7,1,1,-172,-3.3862,-3.5359,-3.1586,4.7090,-2.4668,2.7920,...,1.0243,1.0963,4.9345,-2.2770,4.9263,7.2703,-4.4346,14.6373,-3.4949,7.0
109,8,1,1,-168,-2.1085,-1.6439,-1.4426,3.5299,-2.6505,2.2707,...,0.5447,0.3008,4.0784,-1.1520,3.9824,4.7820,-0.3519,11.8092,-7.8560,7.0


In [0]:
#df_one_hot_encoded = pd.get_dummies(df)
#df = df_one_hot_encoded


In [4]:
df_1 = df.groupby(np.arange(len(df))//5).mean()
df_1['High Value'] = 0
df_1.loc[df_1['Value'] >5, 'High Value'] = 1
df_1.head(60)

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value,High Value
0,2,109,1,1,-192,-5.28194,-11.45834,-7.01592,4.34646,-0.61630,...,6.59812,12.31556,1.82132,11.35598,29.65604,14.63076,15.16332,4.32744,7.0,1
1,7,109,1,1,-172,-3.30512,-4.24572,-3.08738,4.48576,-2.22038,...,0.35068,5.34666,-2.06600,5.01884,4.57928,-3.95586,7.91916,-10.30716,7.0,1
2,12,109,1,1,-152,1.59706,-0.09966,0.07688,-0.75786,0.76062,...,-3.46032,1.30698,-2.50444,-0.65652,3.87014,-3.00652,10.36334,-5.29202,7.0,1
3,17,109,1,1,-132,-2.37426,-4.17788,-3.37598,-10.31230,-6.73560,...,-8.76562,-3.48508,-3.80188,-5.81284,4.51054,-3.38320,10.17816,-4.24218,7.0,1
4,22,109,1,1,-112,-2.17000,-0.78998,-0.60840,-10.47876,-9.84046,...,-8.02154,-0.12472,-3.03720,-2.83254,-12.90254,-5.25396,-4.72924,-7.53386,7.0,1
5,27,109,1,1,-92,-2.16822,-0.15722,-1.83916,-6.41194,-4.44880,...,-8.15194,-5.72468,-4.45618,-6.49330,-6.49864,5.64566,-0.65954,9.65366,7.0,1
6,32,109,1,1,-72,5.35254,11.91934,7.04242,4.32732,3.49922,...,-0.56322,-3.81430,-1.07496,-6.31320,-13.88574,-0.58984,-14.30798,4.00562,7.0,1
7,37,109,1,1,-52,6.68590,6.02208,6.65460,11.00398,9.28094,...,11.32264,-0.75678,7.24554,4.33170,-2.64122,-0.15280,-9.54292,3.98722,7.0,1
8,42,109,1,1,-32,-3.30562,-2.79192,-3.05446,-2.22284,0.31164,...,4.19190,-4.12412,0.50110,-1.91676,-3.94708,-0.45596,-14.57106,3.49834,7.0,1
9,47,109,1,1,-12,3.70966,3.93454,3.43382,3.51528,8.01966,...,4.97018,-2.06276,3.07850,2.04520,-2.74082,-3.47826,0.18676,1.90292,7.0,1


In [5]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[120, 179, 57, 158, 123, 175, 174, 40, 11, 113, 117, 68, 54, 144, 62, 171, 156, 64, 185, 13, 181, 127, 6, 103, 59, 74, 187, 194, 77, 53, 19, 169, 39, 48, 163, 55, 112, 43, 107, 131, 79, 66, 12, 105, 155, 65, 143, 157, 154, 22, 14, 134, 198, 122, 200, 31, 17, 32, 52, 135, 115, 139, 56, 128, 51, 138, 124, 91, 15, 46, 75, 191, 28, 114, 167, 25, 69, 121, 196, 177, 199, 190, 108, 133, 109, 87, 42, 116, 92, 33, 9, 90, 30, 23, 165, 126, 146, 170, 141, 145]
[1, 2, 3, 4, 5, 7, 8, 10, 16, 18, 20, 21, 24, 26, 27, 29, 34, 35, 36, 37, 38, 41, 44, 45, 47, 49, 50, 58, 60, 61, 63, 67, 70, 71, 72, 73, 76, 78, 80, 81, 82, 83, 84, 85, 86, 88, 89, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 106, 110, 111, 118, 119, 125, 129, 130, 132, 136, 137, 140, 142, 147, 148, 149, 150, 151, 152, 153, 159, 160, 161, 162, 164, 166, 168, 172, 173, 176, 178, 180, 182, 183, 184, 186, 188, 189, 192, 193, 195, 197]


In [0]:
rfc = RandomForestClassifier(n_estimators=500, max_features='auto')
#rfc.fit(x_pd, y_pd);

In [0]:
df_1 = df_1.drop(['Unnamed: 0','Subject'], axis=1)
#df_2.head()

In [0]:
from sklearn.model_selection import GroupShuffleSplit
n_splits = 20
cv = GroupShuffleSplit(n_splits=n_splits, test_size=0.2)
#df_2 = df_1[(df_1['Value']==1) | (df_1['Value']==9)]
#df_2 = df_2.drop(['O1','O2','OZ','FP1','FP2','F3','FZ','F4','P8','TP8','P7','FC4','FT8','TP7','FCZ','F8','F7','FT7','VEOU','VEOL','HEOR','HEOR'], axis=1)
df_2 = df_1.drop(['VEOU','VEOL','HEOR','HEOL'], axis=1)
#df_2 = df_2[df_2['Time'] < 800]
#df_2 = df_2[df_2['Time'] > 400]
r_in = np.zeros(n_splits)
r_out = np.zeros(n_splits)
for i,(train, test) in enumerate(cv.split(df_2, groups=df_2['Trial'])):
  #train = df_2.index[train]
  #test = df_2.index[test]
  y = df_2['High Value']
  X = df_2.drop(['High Value','Value'] axis=1).astype('float')
  rfc.fit(X.iloc[train].values, y.iloc[train])
  predicted = rfc.predict(X.iloc[train])
  actual = y.iloc[train]
  r_in[i] = np.corrcoef(predicted, actual)[0, 1]
  t_predicted = rfc.predict(X.iloc[test])
  t_actual = y.iloc[test]
  r_out[i] = np.corrcoef(t_predicted, t_actual)[0, 1]
  print(r_in[i], r_out[i])
print("Mean R_out for entire time series = %.3g +/- %.3g" % (r_out.mean(), r_out.std()))


1.0 1.0
1.0 1.0
1.0 0.9999999999999999
1.0 1.0
1.0 1.0


In [0]:
df_109 = df.groupby(np.arange(len(df_109))//5).mean()
df_109['High Value'] = 0
df_109.loc[df_109['Value'] >5, 'High Value'] = 1
df_109.head(60)


ValueError: ignored

In [0]:
plt.scatter(t_predicted,predicted)

In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [0]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
